# Practica 2: Convolutional Neural Network (CNN)
*Elaborado por:

Luis Fernando Becerra, BEDA Aprendizaje de Máquinas 2024-1S - 2025-1S

Andres Esteban Marin Manco, BEDA Aprendizaje de Máquinas 2025-1S*

# <b>Clasificación de Imágenes con Redes Neuronales Convolucionales (CNN)</b></font>

Esta guía muestra paso a paso cómo entrenar una **Red Neuronal Convolucional (CNN)** para resolver un problema clásico de visión por computador: la **clasificación de imágenes**.

El objetivo es utilizar una CNN para clasificar imágenes del conjunto de datos **CIFAR-10**, el cual contiene 60,000 imágenes a color, de tamaño 32x32 píxeles, distribuidas en 10 categorías como: aviones, gatos, automóviles, perros, etc.

<br>

## <b>¿Cómo lo haremos?</b></font>

Usaremos la **API Secuencial de Keras**, una interfaz de alto nivel provista por TensorFlow que permite construir modelos de manera muy intuitiva. Gracias a esta API, podemos:

- **Definir la arquitectura de la red** en unas pocas líneas de código.
- **Compilar y entrenar el modelo** de forma eficiente.
- **Evaluar el desempeño del modelo** con funciones predefinidas.
- **Visualizar resultados** fácilmente con herramientas como Matplotlib.

<br>

## <b>¿Qué aprenderás?</b></font>
A lo largo de esta guía aprenderás a:

- Cargar y preprocesar datos de imágenes.
- Construir una arquitectura CNN desde cero.
- Entrenar el modelo y monitorear su rendimiento.
- Evaluar los resultados obtenidos y realizar predicciones.

<br>

<p align="center">
  <img src="https://i0.wp.com/developersbreach.com/wp-content/uploads/2020/08/cnn_banner.png?fit=1400%2C658&ssl=1" width="800"/>
</p>



### Importar librerias

In [ ]:
##
import matplotlib.pyplot as plt
####

## <b>Descarga y preparación del conjunto de datos CIFAR-10</b></font>

El conjunto de datos **CIFAR-10** contiene **60.000 imágenes en color**, de tamaño **32x32 píxeles**, distribuidas en **10 clases diferentes**, como aviones, gatos, automóviles, camiones, entre otros.

Cada clase tiene exactamente **6.000 imágenes**. El conjunto de datos se divide en:

- **50.000 imágenes de entrenamiento**
- **10.000 imágenes de prueba**

Todas las clases son **mutuamente excluyentes**, lo que significa que cada imagen pertenece a una única categoría.

## <b>Cargar los datos en memoria</b></font>

Puedes cargar el conjunto de datos directamente desde la API de Keras usando la siguiente instrucción:



In [ ]:
#Cargar datos entrenamiento y prueba
###

Un paso importante de preprocesamiento de los datos a usar con redes profundas es la normalización. Este paso ayuda a los optimizadores a converger más rapido, sin que ninguna de las caracteristicas tenga mayor peso en la selección de la dirección de la convergencia. En el caso de las imágenes en escala de grises y RGB, la normalización es muy sencilla. Usualmente las imágenes en escala de grises (1 sola capa) o RGB (3 capas correspondientes a los colores rojo, verde y azul) se encuentran en formato UINT8, es decir, que cada prixel se representa por valores de 0 a 255. Por lo tanto, para la normalización de los datos solo requerimos dividir por 255.

Nota: cuando se trabajan con otros tipos de imágenes (por ejemplo satelitales) es importante revisar el formato de los datos, o realizar una normalización MIN-MAX.

In [ ]:
#Normalización de los datos
###

## <b>Verificar los datos</b></font>

Para asegurarnos de que el conjunto de datos se ha cargado correctamente, visualizamos las **primeras 25 imágenes de entrenamiento** junto con su **etiqueta(nombre) de clase**.

Esto nos permite:
- Confirmar que las imágenes están bien formateadas.
- Ver que las etiquetas corresponden correctamente a su clase.

Cada imagen se muestra en una cuadrícula de 5x5 con su nombre de clase debajo.

In [ ]:
#Visualización de los datos
#Nombre de las clases
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']
#Visualización de las imagenes
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays,
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
plt.show()

##  Crear la arquitectura

Vamos a definir una arquitectura común usando una pila de capas convolucionales (Conv2D) y de pooling (MaxPooling2D).

Como entrada, una CNN toma tensores de la forma pixeles_por_fila X pixels_por_columna X no_canales (image_height, image_width, color_channels). Las imágenes en escala de grises solo cuentan con 1 canal, en cambio las imágenes que capturamos con nuestros celulares o camaras convencionales usualmente tienen 3 canales correspondientes al RGB.

En nuestro ejemplo la dimesión de los datos es de 32x32x3. Con estas dimensiones configuramos la capa de entrada.

In [ ]:
#Definición del modelo

#Capa convolucional y pooling

#Capa convolucional y pooling

#Capa convolucional


Recordemos que con el metodo summary() podemos revisar la arquitectura de nuestra red, así como ver el número de parámetros que debemos entrenar.

In [ ]:
##

En el resumen del modelo deberias ver que la salida de cada capa Conv2D y MaxPooling2D es un tensor 3D de forma (altura, anchura, canales). Las dimensiones de anchura y altura tienden a reducirse a medida que se profundiza en la red, esto por el efecto del pooling. El número de canales de salida para cada capa Conv2D se controla mediante el primer argumento (por ejemplo, 32 o 64). Normalmente, a medida que la anchura y la altura se reducen, puedes permitirte (computacionalmente) añadir más canales de salida en cada capa Conv2D.

Para completar el modelo, vamos a introducir el último tensor de salida (de la forma (4, 4, 64)) en una o más capas Densas para realizar la clasificación. Las capas densas toman vectores como entrada (que son 1D), mientras que la salida actual es un tensor 3D. Por lo cual, es nesario en primer lugar "aplanar" el tensor (Flatten()), y a continuación, añadir una o más capas densas. CIFAR tiene 10 clases de salida, por lo que se utiliza una capa Densa final con 10 salidas.

Recuerda: El número de capas convolucionales y densas, así como el número de filtros convolucionales, tipo de pooling, número de neuronas ocultas, funciones de activación, son parametros que debe determinar el ingeniero ML.

Visualiza la arquitectura completa de tu modelo:

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model,
           show_shapes=True,
           show_layer_names=True,
           dpi=50,
           #rankdir="LR" #LR:plotea horizontal, TB:plotea hacia abajo (por defecto)
)

##<b>Entendiendo el resumen del modelo</b></font>

En el resumen del modelo (`model.summary()`), puedes observar que la salida de cada capa `Conv2D` y `MaxPooling2D` es un **tensor 3D** con forma:


| Capa             | Tipo          | Forma de salida          | Parámetros |
|------------------|---------------|---------------------------|------------|
| `conv2d`         | Conv2D        | (None, 30, 30, 32)        | 896        |
| `max_pooling2d`  | MaxPooling2D  | (None, 15, 15, 32)        | 0          |
…
<br

## ¿Qué ocurre a medida que profundizamos en la red?</b></font>

- **Alto y ancho**: Tienden a **disminuir** por efecto de las convoluciones y el `MaxPooling`.
- **Número de canales (depth)**: Se **incrementa** progresivamente para capturar más características complejas.

Esto responde a una práctica común: a medida que reducimos el tamaño espacial de las imágenes (menos ancho y alto), podemos **aumentar la profundidad** (más filtros) sin que el costo computacional se dispare.

Por ejemplo:
- Una capa `Conv2D(32, ...)` genera 32 mapas de características.
- Una capa `Conv2D(64, ...)` genera 64 mapas, lo cual permite aprender representaciones más ricas.

<br>

Este patrón (reducir resolución y aumentar profundidad) es típico en arquitecturas CNN eficientes y escalables.


## <b>Añadir capas densas al modelo</b></font>

Para completar el modelo, debemos **transformar la salida de la base convolucional** en una representación que pueda ser interpretada por una capa de clasificación.

Actualmente, la salida es un tensor de forma **(4, 4, 64)**, es decir, un volumen tridimensional.  
Las **capas densas (`Dense`) requieren vectores 1D**, así que primero debemos **aplanar** este tensor con una capa `Flatten`.

<br>

## <b>Estructura final</b></font>

1. **`Flatten`**: Convierte (4, 4, 64) → 1024 unidades.
2. **`Dense(64)`**: Capa oculta totalmente conectada con activación ReLU.
3. **`Dense(10)`**: Capa de salida con 10 unidades (una por cada clase del CIFAR-10).

<br>


_<font color="purple"><b>Imagen de referencia de transformación a capa flattened</b></font>_

<p align="center">
  <img src="https://towardsdatascience.com/wp-content/uploads/2022/06/1N74xher1f5gJSHY-_o_rQQ.png" width="200"/>
</p>


In [ ]:
#Codigo para agregar capas
##

## Visualiza la arquitectura completa de tu modelo</b></font>

In [ ]:
model.summary()

In [ ]:
plot_model(model,
           show_shapes=True,
           show_layer_names=True,
           dpi=50,
           #rankdir="LR" #LR:plotea horizontal, TB:plotea hacia abajo (por defecto)
)

El resumen del modelo muestra que la salida de la última capa convolucional, con forma **(4, 4, 64)**, fue **aplanada** por la capa `Flatten` a un vector de **1024 unidades** antes de pasar por dos capas densas.

Este vector 1D fue luego procesado por:

- Una capa densa intermedia con **64 neuronas** y activación `ReLU`.
- Una **capa de salida con 10 neuronas**, correspondiente a las **10 clases** del conjunto CIFAR-10.

## <b>Compile y entrene el modelo</b></font>

<p align="center">
  <img src="https://i.gifer.com/origin/b4/b4d657e7ef262b88eb5f7ac021edda87.gif" width="80"/>
</p>

> ⚠️ <font color="red"><b>Oops... ¿entrenando en CPU?</b></font>

Entrenar redes neuronales en CPU puede ser **considerablemente más lento**, especialmente con datasets como CIFAR-10 (imágenes).

<p align="center">
  <img src="https://media.tenor.com/NCp49Vx5_P0AAAAM/spanky-waiting.gif"/>
</p>




<br>

**Recomendación:** Si estás usando Google Colab, activa la **GPU** y ahorra tiempo:

1. Ve a `Entorno de ejecución` → `Cambiar tipo de entorno de ejecución`.
2. En "Acelerador por hardware", selecciona **GPU T4**.
3. Vuelve a ejecutar toooooodas las celdas.

In [ ]:
#Función de perdida

#Parametros para entrenamiento

#Entrenamiento


### Evalúe el modelo

In [ ]:
#Visualización de resultados de entrenamiento y validación
epochs = range(1, len(history.history["accuracy"]) + 1)
plt.figure(figsize=(12, 5))

#Grafica accuracy
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history["accuracy"], label='Entrenamiento')
plt.plot(epochs, history.history["val_accuracy"], label='Validación')
plt.xlabel('Época')
plt.ylabel('Precisión')
plt.ylim([0.5, 1])
plt.title('Precisión por época')
plt.legend()

#Graficar loss
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history["loss"], label='Entrenamiento')
plt.plot(epochs, history.history["val_loss"], label='Validación')
plt.xlabel('Época')
plt.ylabel('Pérdida (Loss)')
plt.title('Pérdida por época')
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print(test_acc)

## <b>Evaluación del modelo</b></font>

Nuestra sencilla CNN debería haber alcanzado una **precisión en los datos de prueba superior al 70%**, lo cual es un resultado bastante sólido considerando que se construyó con apenas unas pocas líneas de código utilizando la API secuencial de Keras.

Esto demuestra que incluso arquitecturas simples pueden lograr **buen rendimiento en tareas básicas de clasificación de imágenes**, como CIFAR-10.



### ¿Quieres explorar otro enfoque?
## <b></b></font>

Para una implementación más flexible y detallada, puedes revisar el ejemplo de TensorFlow:

**[TensorFlow 2 Quickstart for Experts](https://www.tensorflow.org/tutorials/quickstart/advanced)**

Este enfoque utiliza:

- La **API de subclases de Keras**, que permite definir arquitecturas más personalizadas.
- El ciclo de entrenamiento manual con **`tf.GradientTape`**, ideal para aprender cómo funciona el entrenamiento paso a paso.

Recomendado si buscas mayor control y aprendizaje más profundo sobre el funcionamiento interno de TensorFlow.

In [ ]:
#Funciones de sklearn para metricas de desempeño
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report
from sklearn.metrics import cohen_kappa_score
import numpy as np

In [ ]:
#Predicción
pred_labels = model.predict(test_images)
y = np.argmax(pred_labels,axis=1)

In [ ]:
# Métricas de desempeño
cm_cnn = confusion_matrix(test_labels, y)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_cnn, display_labels=class_names)
disp.plot()
print(classification_report(test_labels, y))
print("kappa: " + str(cohen_kappa_score(test_labels, y)))

# ¡Gran trabajo!